In [1]:
import pandas as pd

df = pd.read_csv("./data/raw/train_cax.csv")
oof = pd.read_csv("./data/result/oof.csv")

dt = pd.read_csv("./data/raw/test_cax.csv")
ds = pd.read_csv("./data/result/LSTM_submit.csv")

dt["label"] = list(ds["label"])
df["pred"] = oof["pred_label"]

In [2]:
from math import floor

df["leak"] = df["GR"]*100
df["leak"] = df["leak"] - df["leak"].apply(floor)
df["leak"] = df["leak"].apply(lambda x: round(x, 11))

for i in range(5):
    for j in range(i+1,5):
        a = set(df[df["label"]==i]["leak"].value_counts().keys())
        b = set(df[df["label"]==j]["leak"].value_counts().keys())
        print(i,j,"\t",len(a),len(b),"\t",len(a&b))

0 1 	 1860 174921 	 0
0 2 	 1860 16100 	 0
0 3 	 1860 155569 	 0
0 4 	 1860 154186 	 0
1 2 	 174921 16100 	 0
1 3 	 174921 155569 	 0
1 4 	 174921 154186 	 0
2 3 	 16100 155569 	 0
2 4 	 16100 154186 	 0
3 4 	 155569 154186 	 0


In [3]:
s = set()

for i in range(5):
    for j in range(i+1,5):
        a = set(df[df["pred"]==i]["leak"].value_counts().keys())
        b = set(df[df["pred"]==j]["leak"].value_counts().keys())
        print(i,j,"\t",len(a),len(b),"\t",len(a&b))
        s |= a&b
        
len(s)

0 1 	 1984 175178 	 99
0 2 	 1984 16654 	 6
0 3 	 1984 155940 	 13
0 4 	 1984 154706 	 25
1 2 	 175178 16654 	 158
1 3 	 175178 155940 	 385
1 4 	 175178 154706 	 374
2 3 	 16654 155940 	 242
2 4 	 16654 154706 	 273
3 4 	 155940 154706 	 302


1775

In [4]:
1-len(df[df["label"]!=df["pred"]])/len(df)

0.9962956818181818

In [5]:
from tqdm import tqdm_notebook as tqdm
d = dict()
for c in tqdm(list(s)):
    k = df[df["leak"]==c]["pred"].value_counts().keys()[0]
    d[c] = k

In [6]:
res = []
for k, v in tqdm(zip(df["leak"], df["pred"]), total=len(df)):
    if k in d:
        res.append(d[k])
    else:
        res.append(v)

In [7]:
df["res"] = res
print(1-len(df[df["label"]!=df["res"]])/len(df))

0.9967902272727273


In [8]:
dt["leak"] = dt["GR"]*100
dt["leak"] = dt["leak"] - dt["leak"].apply(floor)
dt["leak"] = dt["leak"].apply(lambda x: round(x, 11))

ss = set()

for i in range(5):
    for j in range(i+1,5):
        a = set(dt[dt["label"]==i]["leak"].value_counts().keys())
        b = set(dt[dt["label"]==j]["leak"].value_counts().keys())
        print(i,j,"\t",len(a),len(b),"\t",len(a&b))
        ss |= a&b
        
len(ss)

0 1 	 1941 89315 	 69
0 2 	 1941 8425 	 4
0 3 	 1941 77969 	 10
0 4 	 1941 80258 	 6
1 2 	 89315 8425 	 86
1 3 	 89315 77969 	 181
1 4 	 89315 80258 	 167
2 3 	 8425 77969 	 108
2 4 	 8425 80258 	 117
3 4 	 77969 80258 	 139


828

In [9]:
dd = dict()
for c in tqdm(list(ss)):
    k = dt[dt["leak"]==c]["label"].value_counts().keys()[0]
    dd[c] = k

In [10]:
res = []
for k, v in tqdm(zip(dt["leak"], dt["label"]), total=len(dt)):
    if k in dd:
        res.append(dd[k])
    else:
        res.append(v)
        
dt["label"] = res

In [11]:
dt[["unique_id","label"]].to_csv("./data/result/processed.csv", index=False)

In [12]:
from random import choice

idx = df[df["label"]!=df["res"]].index
k = choice(list(idx))

df.iloc[k-10:k+10,:]

,row_id,well_id,GR,label,pred,leak,res
951256,856,864,65.142939,2,2,0.293899,2
951257,857,864,54.262939,2,2,0.293899,2
951258,858,864,65.772939,2,2,0.293899,2
951259,859,864,44.322939,2,2,0.293899,2
951260,860,864,86.052939,2,2,0.293899,2
951261,861,864,64.412939,2,2,0.293899,2
951262,862,864,67.052939,2,2,0.293899,2
951263,863,864,78.582939,2,2,0.293899,2
951264,864,864,52.572939,2,2,0.293899,2
951265,865,864,102.116431,1,3,0.643093,3
